In [ ]:
    def update_robot_pose(self):
        """ Update the estimate of the robot's pose given the updated particles.
            There are three logical methods for this:
                (1): compute the mean pose
                (2): compute the most likely pose (i.e. the mode of the distribution)
                (3): use clustering algorithms to find the cluster of highest weight

            Our strategy is #2 to enable better tracking of unlikely particles in the future
        """
        # first make sure that the particle weights are normalized
        self.normalize_particles()

        chosen_one = max(self.particle_cloud, key=lambda p: p.w)
        #
        # Using MeanShift scikit learn
        x = [p.x for p in self.particle_cloud]
        y = [p.y for p in self.particle_cloud]
        weights = [p.w for p in self.particle_cloud]
        X = np.array(zip(x, y))

        bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=self.n_particles)
        ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
        ms.fit(X)
        labels = ms.labels_
        cluster_centers = ms.cluster_centers_

        labels_unique = np.unique(labels)
        n_clusters_ = len(labels_unique)

        print("number of estimated clusters : %d" % n_clusters_)

        clusterWeights = {}
        for lable in labels_unique:
            clusterWeights[lable] = 0
        for w, lable in zip(weights, labels):
            clusterWeights[lable] += w

        v = list(clusterWeights.values())
        k = list(clusterWeights.keys())
        cluster_centers[k[v.index(max(v))]]

        # plt.figure(1)
        # plt.clf()
        #
        # colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
        # for k, col in zip(range(n_clusters_), colors):
        #     my_members = labels == k
        #     cluster_center = cluster_centers[k]
        #     plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
        #     plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
        #              markeredgecolor='k', markersize=14)
        # plt.title('Estimated number of clusters: %d' % n_clusters_)
        # plt.show()

        # TODO: assign the lastest pose into self.robot_pose as a geometry_msgs.Pose object
        # just to get started we will fix the robot's pose to always be at the origin
        self.robot_pose = chosen_one.as_pose()